In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

24/06/07 13:51:06 WARN Utils: Your hostname, truongvude resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface wlp4s0)
24/06/07 13:51:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/07 13:51:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/07 13:51:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/07 13:51:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
df_green = spark.read.parquet("data/pq/green/*/*")

In [5]:
df_green.registerTempTable("green")

/home/truongvude/spark/spark-3.5.1-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [6]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [7]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc("hour", lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM 
    green
WHERE
    lpep_pickup_datetime >= "2020-01-01 00:00:00"
GROUP BY 
    1, 2
""")

In [21]:
df_green_revenue \
    .repartition(20) \
    .write.parquet("data/report/revenue/green", mode="overwrite")

24/06/06 17:52:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [22]:
df_yellow = spark.read.parquet("data/pq/yellow/*/*")

In [23]:
df_yellow.registerTempTable("yellow")

/home/truongvude/spark/spark-3.5.1-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [24]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc("hour", tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM 
    yellow
WHERE
    tpep_pickup_datetime >= "2020-01-01 00:00:00"
GROUP BY 
    1, 2
""")

In [11]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 08:00:00| 162| 767.8100000000003|            46|
|2020-01-04 18:00:00| 140|2618.9099999999985|           169|
|2020-01-27 16:00:00|  88|1478.9999999999995|            53|
|2020-01-05 02:00:00| 114| 5647.420000000002|           329|
|2020-01-19 23:00:00|  90|2217.8799999999987|           150|
|2020-01-23 17:00:00| 140| 6545.520000000008|           364|
|2020-01-09 11:00:00| 107|3677.2599999999966|           240|
|2020-01-26 20:00:00| 239|2532.9399999999987|           178|
|2020-01-03 23:00:00| 132| 24519.23999999997|           448|
|2020-01-26 20:00:00| 236|3540.5199999999977|           242|
|2020-01-24 19:00:00| 186|11532.320000000029|           625|
|2020-01-24 03:00:00| 264|             249.6|            11|
|2020-01-31 15:00:00| 143|2763.2299999999977|           182|
|2020-01-23 21:00:00| 14

In [25]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet("data/report/revenue/yellow", mode="overwrite")

24/06/06 17:53:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/06/06 17:53:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [8]:
df_green_revenue = spark.read.parquet("data/report/revenue/green")
df_yellow_revenue = spark.read.parquet("data/report/revenue/yellow/")

In [9]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed("amount", "green_amount") \
    .withColumnRenamed("number_records", "green_number_record")

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed("amount", "yellow_amount") \
    .withColumnRenamed("number_records", "yellow_number_record")

In [10]:
df_joins = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=["hour", "zone"], how="outer")

In [11]:
df_joins.show()

+-------------------+----+-----------------+-------------------+------------------+--------------------+
|               hour|zone|     green_amount|green_number_record|     yellow_amount|yellow_number_record|
+-------------------+----+-----------------+-------------------+------------------+--------------------+
|2020-01-01 00:00:00|   7|769.7299999999997|                 45| 455.1700000000001|                  38|
|2020-01-01 00:00:00|  13|             NULL|               NULL|1214.8000000000002|                  56|
|2020-01-01 00:00:00|  18|              7.8|                  1|               5.8|                   1|
|2020-01-01 00:00:00|  32|68.94999999999999|                  2|              18.0|                   1|
|2020-01-01 00:00:00|  34|             NULL|               NULL|              19.3|                   1|
|2020-01-01 00:00:00|  83|94.09999999999998|                  7|               9.8|                   1|
|2020-01-01 00:00:00|  90|             NULL|           

In [43]:
df_joins.write.parquet("data/report/revenue/total", mode="overwrite")

24/06/06 18:25:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [15]:
df_joins = spark.read.parquet("data/report/revenue/total")

In [16]:
df_joins

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_record: bigint, yellow_amount: double, yellow_number_record: bigint]

In [13]:
df_zones = spark.read.parquet("zones/")

In [14]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [18]:
df_result = df_joins.join(df_zones, df_joins.zone == df_zones.LocationID)

In [21]:
df_result.drop("LocationID", "zone").write.parquet("data/tmp/revenue_zones")

24/06/07 13:58:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
